In [ ]:
import subprocess
import neutralb1.utils as utils

WORKSPACE_DIR = utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

# Final Waveset
This notebook contains the studies for the final waveset that produces the main result of the thesis

Fit Details:
* Only spin-1 ($J=1$) waves
  * {$J^P\ell$} = {$1^+S, 1^-P, 1^+D$}
  * Waves all unconstrained, no D/S ratio or phase parameters
* No isotropic background term
* Uses all 4 polarization orientations
* Split data into bins of mass and $-t$
  * 20 MeV wide bins from $1.0 < M_{\omega\pi^0} < 1.8$ 
* Perform 500 randomized fits in each bin of mass and $-t$ independently
  * Additional 100 bootstrap samples in each bin to improve errors and study correlations

## Setup

In [ ]:
# load common libraries
import pandas as pd
import pickle as pkl
import pathlib
import os, sys
import numpy as np
from uncertainties import ufloat, unumpy, umath
import matplotlib.pyplot as plt
import importlib.resources

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager
import neutralb1.analysis.statistics as nb_stats

style_path = str(importlib.resources.files("neutralb1") / "neutralb1.mplstyle")
plt.style.use(style_path)

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/data-fits/spin-1/no-iso-unconstrained/"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

In [ ]:
%%bash
# print out yaml file used to submit the fits
cat $STUDY_DIR/submission.YAML

In [ ]:
# load in the preprocessed results 
t_bins = ["t_0.10-0.16", "t_0.16-0.23", "t_0.23-0.35", "t_0.35-1.00"]
binned_results = {}
for t_bin in t_bins:
    with open(f"{STUDY_DIR}/{t_bin}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
        data = pkl.load(f)
        binned_results[t_bin] = ResultManager(**data)

binned_results: dict[str, ResultManager]

In [ ]:
for t_bin, results in binned_results.items():
    print(f"Results for t bin: {t_bin}")
    results.summary()

## Standard Plots
First we'll take a look at the typical plots that describe the general features of the model

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(25, 5))
for i, (t_bin, results) in enumerate(binned_results.items()):    
    results.plot.intensity.jp(ax=axs[i])
    t_low, t_high = results.get_t_edges()    
    axs[i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV²", fontsize=16)
    if i != 0:
        axs[i].set_ylabel("")
        axs[i].get_legend().remove()
plt.savefig(f"{STUDY_DIR}/jp_all_t_bins.pdf")    

The below plots can't be aggregated easily across t bins, so just plot for each t bin. They're already saved in the study dir, so just plt.show() them here

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.intensity.waves()
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)    
    plt.show()

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.intensity.waves(fractional=True)
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)    
    plt.show()

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.diagnostic.matrix()
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)    
    plt.show()

### Moments

First, just the significant moments

In [ ]:
sig_moments = set()
for t_bin, results in binned_results.items():    
    sig_moments = sig_moments.union(results.get_significant_moments(threshold=0.02))

fig, axs = plt.subplots(
    len(sig_moments), 4, figsize=(20, 25), layout="constrained", sharex=True, sharey="row"
)

row_max = {}
bin_width = 0
for col, (t_bin, results) in enumerate(binned_results.items()):    
    for row, moment in enumerate(sig_moments):        
        ax = axs[row, col]
        
        my_kwargs = {moment: {"label": ""}}
        results.plot.intensity.plot([moment], ax=ax, col_kwargs=my_kwargs)        

        # only put y-label on the first column, and only put the moment name there
        if col != 0:
            ax.set_ylabel("")
        else:
            ax.set_ylabel(utils.convert_moment_name(moment))

        # only put x-label on the last row
        if row != len(sig_moments) - 1:
            ax.set_xlabel("")
        else:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)")

        # track max value for each row to set the same y-lims across columns
        max_val = ax.get_ylim()[1]
        if row not in row_max or max_val > row_max[row]:
            row_max[row] = max_val
    
    # label first row with t-bin info
    t_low, t_high = results.get_t_edges()
    axs[0,col].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$")

    # get for y title
    bin_width = results.get_mass_bin_width()

# set the same y-lims across columns for each row, and save the figure
for col, (t_bin, results) in enumerate(binned_results.items()):
    for row, moment in enumerate(sig_moments):
        axs[row, col].set_ylim(top=row_max[row])              


fig.supylabel(f"Moment Value / {bin_width:.3f} GeV", fontsize=20)

plt.savefig(f"{STUDY_DIR}/significant_moments_all_t_bins.pdf")

Now lets plots all moments, but group them into a multi page pdf. Each page will consist of a 12 x 4 grid

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

all_moments = set()
for t_bin, results in binned_results.items():
    all_moments = all_moments.union(results.moments)

# convert to list to have a deterministic order for the plots
all_moments = sorted(list(all_moments))

with PdfPages(f"{STUDY_DIR}/all_moments_all_t_bins.pdf") as pdf:
    # iterate through the moments in chunks of 12, since we can only fit 12 rows on a page
    for i in range(0, len(all_moments), 12):
        moment_chunk = all_moments[i:i+12]

        fig, axs = plt.subplots(
            12, 4, 
            figsize=(20, 25), layout="constrained", 
            sharex=True, sharey="row"
        )
        for col, (t_bin, results) in enumerate(binned_results.items()):        
            for row, moment in enumerate(moment_chunk):
                ax = axs[row, col]
                
                my_kwargs = {moment: {"label": ""}}

                results.plot.intensity.plot([moment], ax=ax, col_kwargs=my_kwargs)

                ax.grid(True, alpha=0.8)

                # only put y-label on the first column, and only put the moment name there
                if col != 0:
                    ax.set_ylabel("")
                else:
                    ax.set_ylabel(utils.convert_moment_name(moment))

                # only put x-label on the last row
                if row != len(moment_chunk) - 1:
                    ax.set_xlabel("")
                else:
                    ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)")

                # label first row with t-bin info
                if row == 0:
                    t_low, t_high = results.get_t_edges()
                    ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$")

            # get for y title, we assumed its the same across all pages
            bin_width = results.get_mass_bin_width()

        # remove extra subplots if we have less than 12 moments on this page
        if len(moment_chunk) < 12:
            for row in range(len(moment_chunk), 12):
                for col in range(4):
                    axs[row, col].set_visible(False)

        fig.supylabel(f"Moment Value / {bin_width:.3f} GeV", fontsize=20)
        pdf.savefig(fig)
        plt.close(fig)

print(f"PDF saved to: {STUDY_DIR}/all_moments_all_t_bins.pdf")


### Wave Fit Fractions Across $-t$

In [ ]:
fig, axs = plt.subplots(
    2, 3, 
    figsize=(18, 9), layout="constrained", 
    sharex=True, sharey="row"
)

col_l_vals = ["S", "P", "D"]
col_jp_vals = ["1p", "1m", "1p"]
for col, (l_val, jp_val) in enumerate(zip(col_l_vals, col_jp_vals)):
    for row, (refl, color) in enumerate(zip(["p", "m"], ["red", "blue"])):
        ax = axs[row, col]

        # plot the average fractional contributions of the 3 m projections for the given wave in each plot as a function of t
        for m, marker in zip(["m", "0", "p"], ["v", "o", "^"]):
            m_map = {"m": "-1", "0": "0", "p": "+1"}
            wave = f"{refl}{jp_val}{m}{l_val}"
    
            # since mass regions change so drastically in b1 and rho regions, we'll split the mass range in half 
            wave_fractions_low_mass = []
            wave_fraction_errs_low_mass = []
            t_avg_low_mass = []
            t_rms_low_mass = []

            wave_fractions_high_mass = []
            wave_fraction_errs_high_mass = []
            t_avg_high_mass = []
            t_rms_high_mass = []

            for t_bin, results in binned_results.items():
                low_mass_indices = results.get_fit_indices(1.0, 1.4)
                high_mass_indices = results.get_fit_indices(1.4, 1.8)

                for i, indices in enumerate([low_mass_indices, high_mass_indices]):

                    wave_value = results.fit_df[wave].loc[indices]
                    wave_err = results.plot.intensity.get_bootstrap_error(wave).loc[indices]
                    intensity = results.fit_df["generated_events"].loc[indices]
                    intensity_err = results.plot.intensity.get_bootstrap_error("generated_events").loc[indices]

                    fit_fraction = unumpy.uarray(wave_value, wave_err) / unumpy.uarray(intensity, intensity_err)
                    weighted_avg_fit_fraction, weighted_avg_fit_fraction_err = nb_stats.weighted_average(unumpy.nominal_values(fit_fraction), unumpy.std_devs(fit_fraction), scale=False) # type: ignore

                    if i == 0:
                        wave_fractions_low_mass.append(weighted_avg_fit_fraction)
                        wave_fraction_errs_low_mass.append(weighted_avg_fit_fraction_err)
                        t_avg_low_mass.append(results.data_df.loc[indices,"t_avg"].mean())
                        t_rms_low_mass.append(results.data_df.loc[indices,"t_rms"].mean())
                    else:
                        wave_fractions_high_mass.append(weighted_avg_fit_fraction)
                        wave_fraction_errs_high_mass.append(weighted_avg_fit_fraction_err)
                        t_avg_high_mass.append(results.data_df.loc[indices,"t_avg"].mean())
                        t_rms_high_mass.append(results.data_df.loc[indices,"t_rms"].mean())

            ax.errorbar(
                x=t_avg_low_mass, xerr=t_rms_low_mass, y=wave_fractions_low_mass, yerr=wave_fraction_errs_low_mass, 
                marker=marker, label=f"m={m_map[m]} (low mass)", linestyle="", color=color, markersize=10
            )
            ax.errorbar(
                x=t_avg_high_mass, xerr=t_rms_high_mass, y=wave_fractions_high_mass, yerr=wave_fraction_errs_high_mass, 
                marker=marker, markerfacecolor="white", markeredgecolor=color, label=f"m={m_map[m]} (high mass)", linestyle="", color=color, markersize=10
            )
        
        ax.set_xlim(left=0.1)        
        
        if row == 0:
            ax.set_title(utils.convert_amp_name(wave).replace(r"_{+1}", r"_{m}").replace(r"{(+)}", r"{(\varepsilon)}"), fontsize=20)
        else:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)", fontsize=14)
        if col == 0:
            if row == 0:
                ax.set_ylabel(r"Fractional Contribution ($\varepsilon=+1$)", fontsize=14, loc="top")
            else:
                ax.set_ylabel(r"Fractional Contribution ($\varepsilon=-1$)", fontsize=14, loc="top")
            ax.legend(fontsize=12, loc="best")

plt.savefig(f"{STUDY_DIR}/fractional_wave_contributions_by_t.pdf")

## D/S Ratio and Phase

Even though the ratio and phase are not constrained like in past analyses, we can plot the phase difference, and calculate the ratio between the individual wave's amplitude.

In [ ]:
# we only need to track S-waves to remove, since it will remove partner D-wave as well
all_s_waves = [c for c in binned_results[t_bins[0]].coherent_sums["eJPmL"] if c.endswith("S")]

for s_wave in all_s_waves:
    exclude_waves = all_s_waves.copy()
    exclude_waves.remove(s_wave) # we want to keep one DS wave at a time

    fig, axs = plt.subplots(
        3,
        4,
        sharey="row",
        figsize=(20, 10),
        gridspec_kw={"wspace": 0.0, "hspace": 0.12},
        layout="constrained",
    )
    for i, (t_bin, results) in enumerate(binned_results.items()):
        results.plot.diagnostic.ds_ratio(axs = axs[:, i], exclude_waves=exclude_waves)

        for ax in axs[:, i]:
            ax.grid(True, alpha=0.8)

        if s_wave.startswith("p"):            
            axs[0,i].set_yscale("linear")
            axs[0,i].set_ylim(0, 1)
        t_low, t_high = results.get_t_edges()
        axs[0,i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=16)
        if i != 0:
            for ax in axs[:, i]:
                if ax.get_legend() is not None:
                    ax.get_legend().remove()
                ax.set_ylabel("")
    plt.savefig(f"{STUDY_DIR}/ds_ratio_{s_wave}.pdf")

Lets calculate the average ratio and phase for each D/S pair in the same E852 range, and plot it as a function of -t for each pair

In [ ]:
ds_pairs_pos = [["1pmS", "1pmD"], ["1p0S", "1p0D"], ["1ppS", "1ppD"]]

# plot of 2 params (ratio, phase) by 3 columns (DS wave pairs) as function of t in each plot
fig, axs = plt.subplots(
    2, 3,
    sharex=True,
    sharey="row",
    figsize=(15, 7),
    gridspec_kw={"wspace": 0.0, "hspace": 0.1},
    layout="constrained",
)

marker_list = ["v", "o", "^"]
for col, (s_wave_base, d_wave_base) in enumerate(ds_pairs_pos):
    for reflectivity, color in zip(["p", "m"], ["red", "blue"]):
        s_wave = f"{reflectivity}{s_wave_base}"
        d_wave = f"{reflectivity}{d_wave_base}"

        # calculate the average ds ratio and its standard deviation for the E852 mass range in each t bin
        avg_ratios = []
        avg_ratio_errs = []
        avg_phases = []
        avg_phases_errs = []
        t_avg = []
        t_rms = []
        for i, (t_bin, results) in enumerate(binned_results.items()):        
            indices = results.get_fit_indices(1.155, 1.315) # E852 mass range

            # ratio
            s_wave_values = results.fit_df.loc[indices, s_wave]
            d_wave_values = results.fit_df.loc[indices, d_wave]
            s_wave_errs = results.plot.intensity.get_bootstrap_error(s_wave).loc[indices]
            d_wave_errs = results.plot.intensity.get_bootstrap_error(d_wave).loc[indices]

            s_wave_uarray = unumpy.uarray(s_wave_values, s_wave_errs)
            d_wave_uarray = unumpy.uarray(d_wave_values, d_wave_errs)

            ratio = unumpy.sqrt(d_wave_uarray / s_wave_uarray) # type: ignore
            ratio_values = unumpy.nominal_values(ratio)
            ratio_errs = unumpy.std_devs(ratio)

            weighted_ratio_avg, weighted_ratio_err = nb_stats.weighted_average(ratio_values, ratio_errs) # type: ignore        
            avg_ratios.append(weighted_ratio_avg)
            avg_ratio_errs.append(weighted_ratio_err)

            # phase
            phase = results.phase_difference_dict[(s_wave, d_wave)]
            phase_values = results.fit_df.loc[indices, phase]        
            phase_errs = results.plot.phase.get_bootstrap_error(phase).loc[indices]

            weighted_phase_avg, weighted_phase_err = nb_stats.weighted_average(phase_values, phase_errs) # type: ignore
            avg_phases.append(weighted_phase_avg)
            avg_phases_errs.append(weighted_phase_err)

            # get the average of RMS and t averages in the mass range
            t_avg_values = results.data_df.loc[indices, "t_avg"]
            t_rms_values = results.data_df.loc[indices, "t_rms"]
            t_avg.append(np.mean(t_avg_values))
            t_rms.append(np.mean(t_rms_values))

        # now plot the average ratios and phases as a function of t, with error bars
        label = utils.convert_amp_name(s_wave).replace("S", "(D/S)").replace(r"{(-)}", r"{(\varepsilon)}")
        axs[0, col].errorbar(
            x=t_avg, xerr=t_rms, y=avg_ratios, yerr=avg_ratio_errs, 
            marker=marker_list[col], linestyle="", color=color, markersize=6,
            label="Natural" if reflectivity == "p" else "Unnatural"
        )
        axs[1, col].errorbar(
            x=t_avg, xerr=t_rms, y=avg_phases, yerr=avg_phases_errs,
            marker=marker_list[col], linestyle="", color=color, markersize=6
        )
        axs[1, col].errorbar(
            x=t_avg, xerr=t_rms, y=np.array(avg_phases)*-1, yerr=avg_phases_errs,
            marker=marker_list[col], linestyle="", color=color, markersize=6
        )

    # plot the e852 ratio and phase as a line with transparent error bands
    axs[0, col].axhline(0.269, color="black", linestyle="-", label="E852")
    axs[0, col].fill_between(
        x=[0, 1.5], y1=0.269-0.019, y2=0.269+0.019, color="black", alpha=0.2
    )
    axs[1, col].axhline(10.54, color="black", linestyle="-")
    axs[1, col].fill_between(
        x=[0, 1.5], y1=10.54-6.3, y2=10.54+6.3, color="black", alpha=0.2
    )

    axs[0, col].set_xlim(0, 1.0)
    axs[1, col].set_xlim(0, 1.0)

    axs[0, col].set_title(rf"{label}", fontsize=16) # type: ignore

    axs[0, col].set_ylim(bottom=0)
    axs[1,col].set_ylim(-180, 180)
    axs[1,col].set_yticks(np.linspace(-180, 180, 5))

    axs[1,col].set_xlabel(r"-t (GeV$^2$)")

    if col == 0:
        axs[0, col].set_ylabel("D/S Ratio", loc="top")
        axs[1, col].set_ylabel("D-S Phase (°)", loc="top")
        axs[0,col].legend()        

plt.savefig(f"{STUDY_DIR}/ds_vs_t.pdf")

## $b_1(1235)$ and $1^{--}$ interference

In [ ]:
fig, axs = plt.subplots(
    2,
    4,
    sharex=True,   
    sharey="row",     
    gridspec_kw={"wspace": 0.0, "hspace": 0.07},
    height_ratios=[3, 1],
    figsize=(20, 8),
    layout="constrained",
)

for i, (t_bin, results)in enumerate(binned_results.items()):
    colors = plt.colormaps["Dark2"].colors # type: ignore # match colors to JP plot

    results.plot.phase.mass_phase(
        amp1="p1p0S", amp2="p1mpP",
        amp1_kwargs={"color":colors[2], "alpha": 1.0},
        amp2_kwargs={"color":colors[3], "alpha": 1.0},
        amp_ax=axs[0, i],
        phase_ax=axs[1, i],
    )
    t_low, t_high = results.get_t_edges()
    axs[0,i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=18)    
    if i != 0:
        axs[0,i].set_ylabel("")
        axs[1,i].set_ylabel("")
        axs[0,i].get_legend().remove()

for i in range(4):
    max_val = axs[0,i].set_ylim(top=7e5)

plt.savefig(f"{STUDY_DIR}/mass_phase_all_t_bins.pdf")

Let's view the bootstrap distributions with a joyplot

In [ ]:
# they have to be made individually because the joyplot is a bunch of axes, so we can't easily put them all in one figure
for i, (t_bin, results)in enumerate(binned_results.items()):
    t_low, t_high = results.get_t_edges()
    utils.big_print(f"{t_low} < -t < {t_high}", 2.0)

    colors = plt.colormaps["Dark2"].colors # type: ignore # match colors to JP plot
    colors = list(colors[2:]) # start here so that they match the jp style
    results.plot.bootstrap.joyplot(
        ["p1p0S", "p1mpP"], colormap=colors      
    )
    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/joyplot_p1p0S_p1mpP.pdf")
    plt.show()

In [ ]:
for i, (t_bin, results)in enumerate(binned_results.items()):
    t_low, t_high = results.get_t_edges()
    utils.big_print(f"{t_low} < -t < {t_high}", 2.0)
    phase = results.phase_difference_dict[("p1p0S", "p1mpP")]
    colormap = ["gray"]
    results.plot.bootstrap.joyplot(
        [phase], colormap=colormap
    )
    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/joyplot_p1p0S_p1mpP_phase.pdf")
    plt.show()  

## Naturality Contributions

### Total

In [ ]:
fig, axs = plt.subplots(
    1, 4, 
    figsize=(20, 5), sharey=True, layout="constrained"
)
for i, (t_bin, results) in enumerate(binned_results.items()):
    results.plot.intensity.plot(
        ["p", "m"], 
        fractional=True, 
        col_kwargs={"p": {"color":"red", "label":"Natural"}, "m": {"color":"blue", "label":"Unnatural"}},
        ax=axs[i]
    )
    axs[i].ticklabel_format(style="plain")

    t_low, t_high = results.get_t_edges()
    axs[i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=16)
    
    if i != 0:
        axs[i].set_ylabel("")
        if axs[i].get_legend() is not None:
            axs[i].get_legend().remove()

plt.savefig(f"{STUDY_DIR}/natural_unnatural_all_t_bins.pdf")

### Split by $J^P$

In [ ]:
fig, axs = plt.subplots(
    4, 4, figsize=(20, 12), layout="constrained", sharex=True, sharey="row"
)

for col, (t_bin, results) in enumerate(binned_results.items()):
    row_values = ["p1p", "p1m", "m1p", "m1m"]
    labels = ["Natural $1^+$", "Natural $1^-$", "Unnatural $1^+$", "Unnatural $1^-$"]
    colors = ["tab:red", "tab:red", "tab:blue", "tab:blue"]
    for row in range(4):
        ax = axs[row, col]    
        label = labels[row] if col == 0 else ""
        val = row_values[row]
        my_kwargs = {val: {"label": label, "color": colors[row]}}
        results.plot.intensity.plot([val], ax=ax, col_kwargs=my_kwargs)

        if row == 0:
            t_low, t_high = results.get_t_edges()
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=16)

        if row != 3:
            ax.set_xlabel("")

    if col != 0:
        for row in range(4):
            axs[row, col].set_ylabel("")
            if axs[row, col].get_legend() is not None:
                axs[row, col].get_legend().remove()

plt.savefig(f"{STUDY_DIR}/naturalities_all_t_bins.pdf")

## Statistical Tests

Below we'll read the .txt files, make dictionaries for the normality / bias / correlation test failures, and average across the t bins.
We'll generally skip the phase differences, since they don't directly correlate with other observables, and their normality tests have issues due to the periodicity.



In [ ]:
import re

normality_failures_amps = {}
bias_failures_amps = {}
correlation_failures_amps = {}

for t_bin, results in binned_results.items():
    with open(f"{STUDY_DIR}/{t_bin}/statistical_analysis_log.txt", "r") as f:
        log_content = f.read()

    # Split into sections by the headers
    sections = re.split(
        r"^Normality Test Failures:|^Bias Test Failures:|^Correlation Test Failures:|^Detailed Correlations:",
        log_content, flags=re.MULTILINE
    )

    # The first element is before the first header, so skip it
    # sections[1]: Normality, [2]: Bias, [3]: Correlation, [4]: Detailed Correlations
    if len(sections) > 1:
        normality_section = sections[1].strip()        
        for line in normality_section.splitlines():
            if line.strip():
                key, val = line.split(":", 1)
                wave = key.strip()
                if "_" in wave:
                    continue 
                rate = float(val.strip().replace("%", ""))
                if wave in normality_failures_amps:
                    normality_failures_amps[wave].append(rate)
                else:
                    normality_failures_amps[wave] = [rate]

    if len(sections) > 2:
        bias_section = sections[2].strip()        
        for line in bias_section.splitlines():
            if line.strip():
                key, val = line.split(":", 1)
                wave = key.strip()
                if "_" in wave:
                    continue 
                rate = float(val.strip().replace("%", ""))

                if wave in bias_failures_amps:
                    bias_failures_amps[wave].append(rate)
                else:                    
                    bias_failures_amps[wave] = [rate]

    if len(sections) > 3:
        correlation_section = sections[3].strip()        
        for line in correlation_section.splitlines():
            if line.strip():
                key, val = line.split(":", 1)
                wave = key.strip()
                if "_" in wave:
                    continue 
                rate = float(val.strip().replace("%", ""))
                if wave in correlation_failures_amps:
                    correlation_failures_amps[wave].append(rate)
                else:
                    correlation_failures_amps[wave] = [rate]

# now calculate the average failure rate for each wave across t bins, sort by their average failure rate
normality_failures_amps = dict(sorted(normality_failures_amps.items(), key=lambda item: np.average(item[1]), reverse=True))
bias_failures_amps = dict(sorted(bias_failures_amps.items(), key=lambda item: np.average(item[1]), reverse=True))
correlation_failures_amps = dict(sorted(correlation_failures_amps.items(), key=lambda item: np.average(item[1]), reverse=True))

print("Normality Failures:")
for wave, val in normality_failures_amps.items():
    print(f"{wave}: {np.average(val):.4f}")
print("\nBias Test Failures:")
for wave, val in bias_failures_amps.items():
    print(f"{wave}: {np.average(val):.4f}")
print("\nCorrelation Test Failures:")
for wave, val in correlation_failures_amps.items():
    print(f"{wave}: {np.average(val):.4f}")

In [ ]:
import re

normality_failures_moments = {}
bias_failures_moments = {}
correlation_failures_moments = {}

for t_bin, results in binned_results.items():
    with open(f"{STUDY_DIR}/{t_bin}/statistical_analysis_log.txt", "r") as f:
        log_content = f.read()

    # Split into sections by the headers
    sections = re.split(
        r"^Normality Test Failures \(Moments\):|^Bias Test Failures \(Moments\):|^Correlation Test Failures \(Moments\):|^Detailed Correlations \(Moments\):",
        log_content, flags=re.MULTILINE
    )

    # The first element is before the first header, so skip it
    # sections[1]: Normality, [2]: Bias, [3]: Correlation, [4]: Detailed Correlations
    if len(sections) > 1:
        normality_section = sections[1].strip()        
        for line in normality_section.splitlines():
            if line.strip():
                key, val = line.split(":", 1)
                moment = key.strip()
                rate = float(val.strip().replace("%", ""))

                if moment in normality_failures_moments:
                    normality_failures_moments[moment].append(rate)
                else:
                    normality_failures_moments[moment] = [rate]

    if len(sections) > 2:
        bias_section = sections[2].strip()        
        for line in bias_section.splitlines():
            if line.strip():
                key, val = line.split(":", 1)
                moment = key.strip()
                rate = float(val.strip().replace("%", ""))

                if moment in bias_failures_moments:
                    bias_failures_moments[moment].append(rate)
                else:                    
                    bias_failures_moments[moment] = [rate]

    if len(sections) > 3:
        correlation_section = sections[3].strip()        
        for line in correlation_section.splitlines():
            if line.strip():
                key, val = line.split(":", 1)
                moment = key.strip()
                rate = float(val.strip().replace("%", ""))

                if moment in correlation_failures_moments:
                    correlation_failures_moments[moment].append(rate)
                else:
                    correlation_failures_moments[moment] = [rate]

# now calculate the average failure rate for each wave across t bins, sort by their average failure rate
normality_failures_moments = dict(sorted(normality_failures_moments.items(), key=lambda item: np.average(item[1]), reverse=True))
bias_failures_moments = dict(sorted(bias_failures_moments.items(), key=lambda item: np.average(item[1]), reverse=True))
correlation_failures_moments = dict(sorted(correlation_failures_moments.items(), key=lambda item: np.average(item[1]), reverse=True))

print("Normality Failures:")
for moment, val in normality_failures_moments.items():
    print(f"{moment}: {np.average(val):.4f}")
print("\nBias Test Failures:")
for moment, val in bias_failures_moments.items():
    print(f"{moment}: {np.average(val):.4f}")
print("\nCorrelation Test Failures:")
for moment, val in correlation_failures_moments.items():
    print(f"{moment}: {np.average(val):.4f}")

### Correlations
It's clear that the (m,p)1p0D and (m,p)1p0S waves are overwhelmingly the most often correlated ones. Lets plot their correlation values as a function of mass to look for trends

In [ ]:
fig, axs = plt.subplots(
    1,
    4,    
    sharey="row",         
    figsize=(15, 4),
    layout="constrained",
)
for col, (t_bin, results) in enumerate(binned_results.items()):
    assert results.bootstrap_df is not None, "Bootstrap dataframe is not available in results"

    grouped_df = (
        results.bootstrap_df[["m1p0S", "p1p0S", "m1p0D", "p1p0D", "fit_index"]]
        .groupby("fit_index")
        .corr() # calculate correlations
        .stack()  # reshape to long format
        .reset_index()
        .rename(  # rename columns for clarity
            columns={"level_1": "var1", "level_2": "var2", 0: "correlation"}
        )
    )
    grouped_df = grouped_df.loc[
        (grouped_df["var1"] != grouped_df["var2"]) # remove self-correlations
        & (grouped_df["var1"] < grouped_df["var2"])  # keep only one ordering
        & (grouped_df["var1"].str[-1] == grouped_df["var2"].str[-1]) # we're only interested in correlations between S and D waves of opposite reflectivity
    ]

    s_corrs = grouped_df[
        (grouped_df["var1"] == "m1p0S") & (grouped_df["var2"] == "p1p0S")
    ]["correlation"].values

    d_corrs = grouped_df[
        (grouped_df["var1"] == "m1p0D") & (grouped_df["var2"] == "p1p0D")
    ]["correlation"].values

    mass_bins = results.get_mass_centers()
    
    ax = axs[col]
    ax.plot(mass_bins, s_corrs, marker="o", markersize=6, color="tab:orange", linestyle="", label=utils.convert_amp_name("p1ppS").replace(r"{(+)}", r"{(\pm)}"))
    ax.plot(mass_bins, d_corrs, marker="s", markersize=6, color="tab:purple", linestyle="", label=utils.convert_amp_name("p1ppD").replace(r"{(+)}", r"{(\pm)}"))
    ax.axhline(0, linestyle="-", color="black", alpha=0.6)
    ax.axhline(0.8, linestyle="--", color="red", alpha=0.8)
    ax.axhline(-0.8, linestyle="--", color="red", alpha=0.8)
    t_low, t_high = results.get_t_edges()
    ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=14)
    ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)", loc="right", fontsize=12)
    ax.set_ylim(-1, 1)    
    ax.grid(True)
    if col == 0:
        ax.set_ylabel("Bootstrap Correlation", loc="top", fontsize=12)    
        ax.legend()    

    del grouped_df
plt.savefig(f"{STUDY_DIR}/correlations_all_t_bins.pdf")
    

In [ ]:
for i, (t_bin, results)in enumerate(binned_results.items()):
    t_low, t_high = results.get_t_edges()
    utils.big_print(f"{t_low} < -t < {t_high}", 2.0)
    results.plot.bootstrap.joyplot(
        ["m1p0S", "p1p0S"]        
    )
    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/joyplot_m1p0S_p1p0S.pdf")
    plt.show()

In [ ]:
for i, (t_bin, results)in enumerate(binned_results.items()):
    t_low, t_high = results.get_t_edges()
    utils.big_print(f"{t_low} < -t < {t_high}", 2.0)
    results.plot.bootstrap.joyplot(
        ["m1p0D", "p1p0D"]        
    )
    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/joyplot_m1p0D_p1p0D.pdf")
    plt.show()

### Convergence Rates
View the rate of successful, failed, and converged but with bad error matrix fits

In [ ]:
fig, axs = plt.subplots(
    1,
    4,    
    sharey="row",         
    figsize=(18, 5),
    layout="constrained",
)
for col, (t_bin, results) in enumerate(binned_results.items()):
    my_kwargs = {
        "successful_kwargs": {"markersize":6},
        "failed_kwargs": {"markersize":6},
        "bad_matrix_kwargs": {"markersize":6}
    }
    results.plot.diagnostic.convergence_rates(ax=axs[col], **my_kwargs)
    t_low, t_high = results.get_t_edges()
    axs[col].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=16)
    if col != 0:
        axs[col].set_ylabel("")
        if axs[col].get_legend() is not None:
            axs[col].get_legend().remove()
plt.savefig(f"{STUDY_DIR}/convergence_rates_all_t_bins.pdf")

## Breit-Wigner Lineshape Fits

In [ ]:
from scipy.optimize import curve_fit
from neutralb1.analysis.physics import breit_wigner
vec_bw = np.vectorize(breit_wigner)

# normal breit-wigner, with a tunable scale factor.
# since this Breit-Wigner is defined typically at the amplitude level, we want to view its intensity, so we take |BW|^2
def s_wave_bw_model(mass, scale, bw_mass, bw_width):
    bw = vec_bw(mass, bw_mass=bw_mass, bw_width=bw_width, bw_l=0)    
    return scale * np.square(np.abs(bw))
def p_wave_bw_model(mass, scale, bw_mass, bw_width):
    bw = vec_bw(mass, bw_mass=bw_mass, bw_width=bw_width, bw_l=1)    
    return scale * np.square(np.abs(bw))
def d_wave_bw_model(mass, scale, bw_mass, bw_width):
    bw = vec_bw(mass, bw_mass=bw_mass, bw_width=bw_width, bw_l=2)    
    return scale * np.square(np.abs(bw))

### S-wave

In [ ]:
function_masses = np.arange(1.0, 1.5, 0.001) # centered on b1 region
fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharex=True
)

for col, (t_bin, results) in enumerate(binned_results.items()):
    row_y_titles = [r"$1^+S_{+1}^{(+)}$", r"$1^+S_{0}^{(+)}$", r"$1^+S_{-1}^{(+)}$"]
    row_wave = ["p1ppS", "p1p0S", "p1pmS"]
    for row in range(3):                
        data_masses = results.get_mass_centers()
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values

        # determine where data_mass > 1.5 GeV
        fit_masses = [m for m in data_masses if m < 1.5]
        fit_values = data_values[:len(fit_masses)] 
        fit_errors = data_errors[:len(fit_masses)]       

        popt, pcov = curve_fit(
            s_wave_bw_model,
            fit_masses,
            fit_values,
            sigma=fit_errors,
            p0=[200000, 1.23, 0.14],
        )
        perr = np.sqrt(np.diag(pcov))
        scale_opt, mass_opt, width_opt = popt
        fitted_curve = s_wave_bw_model(function_masses, *popt)

        ax = axs[row, col]
        ax.plot(function_masses, fitted_curve, color="red", linestyle="-", linewidth=2, label="Fitted BW")
        ax.errorbar(
            x=data_masses, 
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker=".", 
            color="black",             
        )
        
        ax.set_ylabel(row_y_titles[row] if col == 0 else "", loc="center", fontsize=18)
        ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

        # Plot PDG shape for comparison
        pdg_mass = 1.2295
        pdg_width = 0.142
        pdg_curve = s_wave_bw_model(function_masses, scale_opt, pdg_mass, pdg_width)
        ax.plot(function_masses, pdg_curve, color="blue", linestyle="--", linewidth=2, alpha=0.6, label="PDG BW")

        # calculate chi squared per degree of freedom for the fit
        residuals = fit_values - s_wave_bw_model(fit_masses, *popt)
        chi_squared = np.sum((residuals / fit_errors) ** 2)
        dof = len(fit_values) - len(popt)
        chi_squared_per_dof = chi_squared / dof
        
        if row == 0:
            t_low, t_high = results.get_t_edges()
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=16)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)")
        
        textstr = (
            f"M: {round(mass_opt*1000)} ± {round(perr[1]*1000)} MeV\n"
            rf"$\Gamma$: {round(width_opt*1000)} ± {round(perr[2]*1000)} MeV"
            "\n"
            rf"$\chi^2$/df: {chi_squared_per_dof:.2f}"
        )
        ax.text(0.95, 0.95, textstr, transform=ax.transAxes, fontsize=12,
                verticalalignment='top', horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.savefig(f"{STUDY_DIR}/BW-b1_s_refl+_all_t_bins.pdf")

### P-wave

In [ ]:
function_masses = np.arange(1.0, 1.8, 0.001) # centered on b1 region
fig, axs = plt.subplots(
    1, 4, figsize=(20, 6), layout="constrained", sharey=True, sharex=True
)
for col, (t_bin, results) in enumerate(binned_results.items()):
    row_y_titles = [r"$1^-P_{+1}^{(+)}$"]
    row_wave = ["p1mpP"]
    for row in range(1):                
        data_masses = results.get_mass_centers()
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values
        
        fit_masses = data_masses
        fit_values = data_values
        fit_errors = data_errors

        popt, pcov = curve_fit(
            p_wave_bw_model,
            fit_masses,
            fit_values,
            sigma=fit_errors,
            p0=[150000, 1.465, 0.4],
        )
        perr = np.sqrt(np.diag(pcov))
        scale_opt, mass_opt, width_opt = popt
        fitted_curve = p_wave_bw_model(function_masses, *popt)

        ax = axs[col]
        ax.plot(function_masses, fitted_curve, color="red", linestyle="-", linewidth=2, label="Fitted BW")
        ax.errorbar(
            x=data_masses, 
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker=".", 
            color="black",             
        )
        
        ax.set_ylabel(row_y_titles[row] if col == 0 else "", loc="center", fontsize=18)
        ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

        # Plot PDG shape for comparison
        pdg_mass = 1.465
        pdg_width = 0.400
        pdg_curve = p_wave_bw_model(function_masses, scale_opt, pdg_mass, pdg_width)
        ax.plot(function_masses, pdg_curve, color="blue", linestyle="--", linewidth=2, alpha=0.6, label="PDG BW")

        # calculate chi squared per degree of freedom for the fit
        residuals = fit_values - p_wave_bw_model(fit_masses, *popt)
        chi_squared = np.sum((residuals / fit_errors) ** 2)
        dof = len(fit_values) - len(popt)
        chi_squared_per_dof = chi_squared / dof
        
        t_low, t_high = results.get_t_edges()
        ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=16)    

        ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)")
        
        textstr = (
            f"M: {round(mass_opt*1000)} ± {round(perr[1]*1000)} MeV\n"
            rf"$\Gamma$: {round(width_opt*1000)} ± {round(perr[2]*1000)} MeV"
            "\n"
            rf"$\chi^2$/df: {chi_squared_per_dof:.2f}"
        )
        ax.text(0.95, 0.95, textstr, transform=ax.transAxes, fontsize=12,
                verticalalignment='top', horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.savefig(f"{STUDY_DIR}/BW-b1_p_refl+_all_t_bins.pdf")

### D-wave

In [ ]:
function_masses = np.arange(1.0, 1.5, 0.001) # centered on b1 region
fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharex=True
)

for col, (t_bin, results) in enumerate(binned_results.items()):
    row_y_titles = [r"$1^+D_{+1}^{(+)}$", r"$1^+D_{0}^{(+)}$", r"$1^+D_{-1}^{(+)}$"]
    row_wave = ["p1ppD", "p1p0D", "p1pmD"]
    for row in range(3):                
        data_masses = results.get_mass_centers()
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values

        # determine where data_mass > 1.5 GeV
        fit_masses = [m for m in data_masses if m < 1.5]
        fit_values = data_values[:len(fit_masses)] 
        fit_errors = data_errors[:len(fit_masses)]       

        popt, pcov = curve_fit(
            d_wave_bw_model,
            fit_masses,
            fit_values,
            sigma=fit_errors,
            p0=[200000, 1.23, 0.14],
        )
        perr = np.sqrt(np.diag(pcov))
        scale_opt, mass_opt, width_opt = popt
        fitted_curve = d_wave_bw_model(function_masses, *popt)

        ax = axs[row, col]
        ax.plot(function_masses, fitted_curve, color="red", linestyle="-", linewidth=2, label="Fitted BW")
        ax.errorbar(
            x=data_masses, 
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker=".", 
            color="black",             
        )
        
        ax.set_ylabel(row_y_titles[row] if col == 0 else "", loc="center", fontsize=18)
        ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

        # Plot PDG shape for comparison
        pdg_mass = 1.2295
        pdg_width = 0.142
        pdg_curve = d_wave_bw_model(function_masses, scale_opt, pdg_mass, pdg_width)
        ax.plot(function_masses, pdg_curve, color="blue", linestyle="--", linewidth=2, alpha=0.6, label="PDG BW")

        # calculate chi squared per degree of freedom for the fit
        residuals = fit_values - d_wave_bw_model(fit_masses, *popt)
        chi_squared = np.sum((residuals / fit_errors) ** 2)
        dof = len(fit_values) - len(popt)
        chi_squared_per_dof = chi_squared / dof
        
        if row == 0:
            t_low, t_high = results.get_t_edges()
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=16)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)")
        
        textstr = (
            f"M: {round(mass_opt*1000)} ± {round(perr[1]*1000)} MeV\n"
            rf"$\Gamma$: {round(width_opt*1000)} ± {round(perr[2]*1000)} MeV"
            "\n"
            rf"$\chi^2$/df: {chi_squared_per_dof:.2f}"
        )
        ax.text(0.95, 0.95, textstr, transform=ax.transAxes, fontsize=12,
                verticalalignment='top', horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.savefig(f"{STUDY_DIR}/BW-b1_d_refl+_all_t_bins.pdf")

## Systematics
Any systematics not covered by our [plot-systematics](../../../scripts/plot_systematics.py) script will be shown here.

### 1D Systematic Groups
Here we plot the systematic results and their barlow significance $\Delta/\sigma_\Delta$ as a function of mass, and the groups of variations together. We save the largest significant residuals to a csv to use in our final systematic uncertainty calculation

In [ ]:
# TODO: FOR ALL below plots, figure out the csv method

In [ ]:
def residual_and_error(
    nominal_result: ResultManager, systematic_result: ResultManager, column: str
) -> tuple[np.ndarray, np.ndarray]:
    """Compute residual and its error for each systematic result in barlow method

    This function computes the residuals (systematic - nominal) for a specified column
    across all mass bins, and returns a list of arrays containing the residuals
    with propagated uncertainties. Note that only MINUIT errors are used.

    Args:
        nominal_result (ResultManager): Best fit result to compare to
        systematic_result ([ResultManager]): systematic result to compare
        column (str): the column in the fit_df to compute residuals for (e.g. "p1p0S")

    Returns:
        tuple[np.ndarray, np.ndarray]: tuple of residual values and errors
    """
    assert (
        column in nominal_result.fit_df.columns
    ), f"Column '{column}' not found in nominal result dataframe"

    # since the columns are amplitudes and phase differences, it is safe
    # to take the absolute value. This is because amplitudes are always positive by
    # definition, and the phase differences have a sign ambiguity in the model that we
    # must account for in the residual calculation
    nominal_values = np.abs(nominal_result.fit_df[column].to_numpy())
    nominal_errors = np.abs(nominal_result.fit_df[f"{column}_err"].to_numpy())

    vectorized_circular_residual = np.vectorize(utils.circular_residual)

    assert (
        column in systematic_result.fit_df.columns
    ), f"Column '{column}' not found in systematic result dataframe"

    sys_values = np.abs(systematic_result.fit_df[column].to_numpy())
    sys_errors = np.abs(systematic_result.fit_df[f"{column}_err"].to_numpy())

    if column in nominal_result.phase_differences:
        # for phase differences, we need to account for its circular nature
        residual = vectorized_circular_residual(
            sys_values, nominal_values, in_degrees=True, low=-180, high=180
        )
    else:
        residual = sys_values - nominal_values
    residual_err = np.sqrt(
        np.abs(np.square(sys_errors) - np.square(nominal_errors))
    )

    # since we had to substitute two nominal fits for failing bins to just get the
    # script to work, we end up with some cases where residual(err) is zero. Zero
    # residual is fine, but zero error causes problems for the plot since we divide
    # by the error. In these cases, we can just set the error to a small value so
    # they show up on the plot zero residual still
    residual_err = np.where(residual_err == 0.0, 1e-6, residual_err)

    return residual, residual_err

In [ ]:
import matplotlib.axes
def plot_1d_residuals(
    nominal_result: ResultManager,
    systematic_result: ResultManager,
    column: str,
    nominal_label: str,
    sys_label: str,
    sys_color: str,
    sys_marker: str,
    amp_ax: matplotlib.axes.Axes,
    res_ax: matplotlib.axes.Axes,
    barlow_ax: matplotlib.axes.Axes,
) -> tuple[matplotlib.axes.Axes, matplotlib.axes.Axes, matplotlib.axes.Axes]:
    
    nominal_values = nominal_result.fit_df[column].to_numpy()
    nominal_errors = nominal_result.fit_df[f"{column}_err"].abs().to_numpy()

    sys_values = systematic_result.fit_df[column].to_numpy()
    sys_errors = systematic_result.fit_df[f"{column}_err"].abs().to_numpy()

    res, res_error = residual_and_error(nominal_result, systematic_result, column)
    normalized_res = res / nominal_values
    normalized_res_error = np.abs(res_error / nominal_values)
    barlow_significance = res / res_error

    mass_bins = nominal_result.get_mass_centers()
    bin_width = nominal_result.get_mass_bin_width()

    amp_ax.errorbar(
        x=mass_bins,
        xerr=bin_width / 2,
        y=nominal_values,
        yerr=nominal_errors,
        marker=".",
        markersize=4,
        color="black",
        alpha=0.8,
        linestyle="",
        label=nominal_label
    )
    amp_ax.errorbar(
        x=mass_bins,
        xerr=bin_width / 2,
        y=sys_values,
        yerr=sys_errors,
        marker=sys_marker,
        markersize=4,
        color=sys_color,
        alpha=0.6,
        linestyle="",
        label=sys_label
    )

    # phases need the negative values plotted to due to sign ambiguity
    if column in nominal_result.phase_differences:
        amp_ax.errorbar(
            x=mass_bins,
            xerr=bin_width / 2,
            y=-nominal_values,
            yerr=nominal_errors,
            marker=".",
            markersize=4,
            color="black",
            alpha=0.8,
            linestyle="",
        )
        amp_ax.errorbar(
            x=mass_bins,
            xerr=bin_width / 2,
            y=-sys_values,
            yerr=sys_errors,
            marker=sys_marker,
            markersize=5,
            color=sys_color,
            alpha=0.6,
            linestyle="",            
        )

    res_ax.errorbar(
        x=mass_bins,
        xerr=bin_width / 2,
        y=res,
        yerr=res_error,
        marker=sys_marker,
        markersize=5,
        color=sys_color,
        alpha=0.6,
        linestyle="",
    )


    barlow_ax.errorbar(
        mass_bins,
        barlow_significance,
        marker=sys_marker,
        markersize=5,
        color=sys_color,
        alpha=0.6,
        linestyle="",
    )
    barlow_ax.axhline(3, color="red", linestyle="--")
    barlow_ax.axhline(-3, color="red", linestyle="--")

    t_low, t_high = nominal_result.get_t_edges()

    amp_ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=18)   
    
    if column in nominal_result.phase_differences:
        amp_ax.set_ylabel(f"Phase Difference (degrees)", loc="top", fontsize=16)
        amp_ax.ticklabel_format(axis="y", style="plain")
        amp_ax.set_ylim(-180, 180)
    else:
        amp_ax.set_ylabel(f"Events / {bin_width:.3f} GeV", loc="top", fontsize=16)
        amp_ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

    res_ax.set_ylabel(r"$\Delta$", loc="center", fontsize=16)

    barlow_ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)", loc="right", fontsize=16)
    barlow_ax.set_ylabel(r"$\Delta/\sigma_\Delta$", loc="center", fontsize=16)

    return amp_ax, res_ax, barlow_ax

#### Event Selection
Since there are so many variations for this, we will split these into subgroups

In [ ]:
chi2_dir = f"{STUDY_DIR}/../systematics/chi2/"
pz_dir = f"{STUDY_DIR}/../systematics/pz/"
shQuality_dir = f"{STUDY_DIR}/../systematics/shQuality/"
unusedE_dir = f"{STUDY_DIR}/../systematics/unusedE/"
unusedTracks_dir = f"{STUDY_DIR}/../systematics/unusedTracks/"

##### Unused Energy

In [ ]:
for col in ["p1p0S", "p1mpP", "p1p0S_p1mpP"]:
    fig, axs = plt.subplots(
        3,
        4,
        sharex=True,   
        sharey="row",     
        gridspec_kw={"wspace": 0.0, "hspace": 0.07},
        height_ratios=[3, 1, 1],
        figsize=(20, 8),
        layout="constrained",
    )

    for value, sys_marker, sys_color in zip([0.11, 0.12], ["^", ">"], ["blue", "green"]):
        unusedE_variation_dir = f"{unusedE_dir}/{value}/"
        systematic_results: dict[str, ResultManager] = {}
        for t_bin in binned_results.keys():
            with open(f"{unusedE_variation_dir}/{t_bin}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
                data = pkl.load(f)
                systematic_results[t_bin] = ResultManager(**data)
        
        nominal_label = r"Nominal (unusedE<0.10)"
        sys_label = rf"unusedE<{value}"
        
        for i, (t_bin, sys_result)in enumerate(systematic_results.items()):
            nominal_result = binned_results[t_bin]

            plot_1d_residuals(
                nominal_result,
                sys_result,
                column=col,
                nominal_label=nominal_label if value == 0.11 else "",
                sys_label=sys_label,
                sys_color=sys_color,
                sys_marker=sys_marker,
                amp_ax=axs[0,i],
                res_ax=axs[1,i],
                barlow_ax=axs[2,i],
            )

            if i != 0:
                axs[0,i].set_ylabel("")            
                axs[1,i].set_ylabel("")
                axs[2,i].set_ylabel("")
            else:
                axs[0,i].legend(fontsize=16)

            fig.suptitle(rf"Systematic Variation: {utils.convert_amp_name(col)}", fontsize=20)
    plt.savefig(f"{unusedE_dir}/systematic_{col}.pdf", bbox_inches="tight", dpi=300)
    plt.show()

##### $P_z^{CM}(\pi^0)$

In [ ]:
for col in ["p1p0S", "p1mpP", "p1p0S_p1mpP"]:
    fig, axs = plt.subplots(
        3,
        4,
        sharex=True,   
        sharey="row",     
        gridspec_kw={"wspace": 0.0, "hspace": 0.07},
        height_ratios=[3, 1, 1],
        figsize=(20, 8),
        layout="constrained",
    )

    for value, sys_marker, sys_color in zip([-0.15, 0.0], ["<", ">"], ["blue", "orange"]):
        pz_variation_dir = f"{pz_dir}/{value}/"
        systematic_results: dict[str, ResultManager] = {}
        for t_bin in binned_results.keys():
            with open(f"{pz_variation_dir}/{t_bin}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
                data = pkl.load(f)
                systematic_results[t_bin] = ResultManager(**data)
        
        nominal_label = r"Nominal ($P_z^{CM}(\pi^0)$>-0.1)"
        sys_label = rf"$P_z^{{CM}}(\pi^0)>{value}$"
        
        for i, (t_bin, sys_result)in enumerate(systematic_results.items()):
            nominal_result = binned_results[t_bin]

            plot_1d_residuals(
                nominal_result,
                sys_result,
                column=col,
                nominal_label=nominal_label if value == 0.0 else "",
                sys_label=sys_label,
                sys_color=sys_color,
                sys_marker=sys_marker,
                amp_ax=axs[0,i],
                res_ax=axs[1,i],
                barlow_ax=axs[2,i],
            )

            if i != 0:
                axs[0,i].set_ylabel("")            
                axs[1,i].set_ylabel("")
                axs[2,i].set_ylabel("")
            else:
                axs[0,i].legend(fontsize=16)

            fig.suptitle(rf"Systematic Variation: {utils.convert_amp_name(col)}", fontsize=20)
    plt.savefig(f"{pz_dir}/systematic_{col}.pdf", bbox_inches="tight", dpi=300)
    plt.show()

##### $\chi^2$

In [ ]:
for col in ["p1p0S", "p1mpP", "p1p0S_p1mpP"]:
    fig, axs = plt.subplots(
        3,
        4,
        sharex=True,   
        sharey="row",     
        gridspec_kw={"wspace": 0.0, "hspace": 0.07},
        height_ratios=[3, 1, 1],
        figsize=(20, 8),
        layout="constrained",
    )

    for value, sys_marker, sys_color in zip([4.0, 6.0], ["<", ">"], ["orange", "blue"]):
        chi2_variation_dir = f"{chi2_dir}/{value}/"
        systematic_results: dict[str, ResultManager] = {}
        for t_bin in binned_results.keys():
            with open(f"{chi2_variation_dir}/{t_bin}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
                data = pkl.load(f)
                systematic_results[t_bin] = ResultManager(**data)
        
        nominal_label = r"Nominal ($\chi^2<5.0$)"
        sys_label = rf"$\chi^2<{value}$"
        
        for i, (t_bin, sys_result)in enumerate(systematic_results.items()):
            nominal_result = binned_results[t_bin]

            plot_1d_residuals(
                nominal_result,
                sys_result,
                column=col,
                nominal_label=nominal_label if value == 4.0 else "",
                sys_label=sys_label,
                sys_color=sys_color,
                sys_marker=sys_marker,
                amp_ax=axs[0,i],
                res_ax=axs[1,i],
                barlow_ax=axs[2,i],
            )

            if i != 0:
                axs[0,i].set_ylabel("")            
                axs[1,i].set_ylabel("")
                axs[2,i].set_ylabel("")
            else:
                axs[0,i].legend(fontsize=16)

            fig.suptitle(rf"Systematic Variation: {utils.convert_amp_name(col)}", fontsize=20)
    plt.savefig(f"{chi2_dir}/systematic_{col}.pdf", bbox_inches="tight", dpi=300)
    plt.show()

##### Unused Tracks

In [ ]:
for col in ["p1p0S", "p1mpP", "p1p0S_p1mpP"]:
    fig, axs = plt.subplots(
        3,
        4,
        sharex=True,   
        sharey="row",     
        gridspec_kw={"wspace": 0.0, "hspace": 0.07},
        height_ratios=[3, 1, 1],
        figsize=(20, 8),
        layout="constrained",
    )

    for value, sys_marker, sys_color in zip([1], [">"], ["blue"]):
        unusedTracks_variation_dir = f"{unusedTracks_dir}/{value}/"
        systematic_results: dict[str, ResultManager] = {}
        for t_bin in binned_results.keys():
            with open(f"{unusedTracks_variation_dir}/{t_bin}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
                data = pkl.load(f)
                systematic_results[t_bin] = ResultManager(**data)
        
        nominal_label = r"Nominal (Unused Tracks = 0)"
        sys_label = rf"Unused Tracks $\leq{value}$"
        
        for i, (t_bin, sys_result)in enumerate(systematic_results.items()):
            nominal_result = binned_results[t_bin]

            plot_1d_residuals(
                nominal_result,
                sys_result,
                column=col,
                nominal_label=nominal_label if value == 1 else "",
                sys_label=sys_label,
                sys_color=sys_color,
                sys_marker=sys_marker,
                amp_ax=axs[0,i],
                res_ax=axs[1,i],
                barlow_ax=axs[2,i],
            )

            if i != 0:
                axs[0,i].set_ylabel("")            
                axs[1,i].set_ylabel("")
                axs[2,i].set_ylabel("")
            else:
                axs[0,i].legend(fontsize=16)

            fig.suptitle(rf"Systematic Variation: {utils.convert_amp_name(col)}", fontsize=20)
    plt.savefig(f"{unusedTracks_dir}/systematic_{col}.pdf", bbox_inches="tight", dpi=300)
    plt.show()

##### Shower Quality

In [17]:
# TODO: finish these

#### Waveset Variations

In [ ]:
for col in ["p1p0S", "p1mpP", "p1p0S_p1mpP"]:
    fig, axs = plt.subplots(
        3,
        4,
        sharex=True,   
        sharey="row",     
        gridspec_kw={"wspace": 0.0, "hspace": 0.07},
        height_ratios=[3, 1, 1],
        figsize=(20, 8),
        layout="constrained",
    )

    result_dirs = [
        f"{STUDY_DIR}../constrained-ratio",
        f"{STUDY_DIR}../negative-ratio",
    ]

    for model_dir, sys_label, sys_marker, sys_color in zip(result_dirs, ["All", "Unnatural"], ["s", "+"], ["purple", "orange"]):        
        systematic_results: dict[str, ResultManager] = {}
        for t_bin in binned_results.keys():
            with open(f"{model_dir}/{t_bin}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
                data = pkl.load(f)
                systematic_results[t_bin] = ResultManager(**data)
        
        nominal_label = r"Final"        
        
        for i, (t_bin, sys_result)in enumerate(systematic_results.items()):
            nominal_result = binned_results[t_bin]

            plot_1d_residuals(
                nominal_result,
                sys_result,
                column=col,
                nominal_label=nominal_label if result_dirs.index(model_dir) == 0 else "",
                sys_label=sys_label,
                sys_color=sys_color,
                sys_marker=sys_marker,
                amp_ax=axs[0,i],
                res_ax=axs[1,i],
                barlow_ax=axs[2,i],
            )

            if i != 0:
                axs[0,i].set_ylabel("")            
                axs[1,i].set_ylabel("")
                axs[2,i].set_ylabel("")
            else:
                axs[0,i].legend(fontsize=16)

            fig.suptitle(rf"Systematic Variation: {utils.convert_amp_name(col)}", fontsize=20)
    plt.savefig(f"{STUDY_DIR}/systematic_waveset_{col}.pdf", bbox_inches="tight", dpi=300)
    plt.show()

### Residuals between mass bin (centers - average) and (bin width - rms)
We only use the bin centers and bin widths for the value and errors, but technically could employ the averages calculated from the events and their rms, as we did for the t bins. Lets tabulate the residuals of these to see if its appreciable

In [ ]:
for i, (t_bin, results)in enumerate(binned_results.items()):
    t_low, t_high = results.get_t_edges()
    utils.big_print(f"{t_low} < -t < {t_high}", 2.0)
    value_residual = (results.data_df["m_center"] - results.data_df["m_avg"]).to_numpy()
    error_residual = (pd.Series(np.full_like(value_residual, 0.02)) - results.data_df["m_rms"]).to_numpy() # fixed bin width of 20 MeV
    
    max_val_res = np.max(np.abs(value_residual))
    max_val_res_idx = np.argmax(np.abs(value_residual))
    max_val_percent_res = max_val_res / results.data_df["m_center"].iloc[max_val_res_idx] * 100

    max_err_res = np.max(np.abs(error_residual))
    max_err_res_idx = np.argmax(np.abs(error_residual))
    max_err_percent_res = max_err_res / results.data_df["m_center"].iloc[max_err_res_idx] * 100

    print(f"Residuals for mass bin centers:")
    print(value_residual)
    print(f"Residuals for mass bin errors:")
    print(error_residual)

    print(f"Max percentage deviation {max_val_percent_res:.2f}% ({max_val_res:.4f} GeV at index {max_val_res_idx}) for mass bin centers")
    print(f"Max percentage deviation {max_err_percent_res:.2f}% ({max_err_res:.4f} GeV at index {max_err_res_idx}) for mass bin errors")


## Cleanup
zip together all the pdfs in each t bin, and zip the pdfs that describe all t bins together

In [ ]:
%%bash
cd ${STUDY_DIR}
zip -u all_t_bin_plots.zip *.pdf
rm single_t_bin_plots.zip
zip -r single_t_bin_plots.zip t_*/plots/*.pdf